In [1]:
# In this scenario we are going to grab the info from excel files.
'''
We need:
1. Prices in different currencies
2. List with Pusblisher names.
3. Clean Publisher Name
4. Check Suppliers, currencies, publishers, discount, shipping costs
5. Check Suppliers special discounts
6. Check restrictions
7. Get landed Cost
8. List of ISBN to ananlyse.

'''

'\nWe need:\n1. Prices in different currencies\n2. List with Pusblisher names.\n3. Clean Publisher Name\n4. Check Suppliers, currencies, publishers, discount, shipping costs\n5. Check Suppliers special discounts\n6. Check restrictions\n7. Get landed Cost\n8. List of ISBN to ananlyse.\n\n'

In [2]:
# 0. Import Libraries and credentials
import pandas as pd
import pyodbc
import numpy as np
import sqlCredentials as sql

In [3]:
# 0. Get databse connection
# Let's pick all the titles gwe can get from the pricing matrix.

prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

prouc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

In [4]:
#1. Prices at different currencies
#this time we are going to use all the titles
list_price_sql = """
SELECT *
FROM Isbn.ListPrice
"""

list_price = pd.read_sql(list_price_sql, prod_db)
list_price.head()

,Id,Isbn,Currency,Price,PriceSource,PriceLastRefreshed,Timestamp
0,1,9780000079145,GBP,67.00,GoldenList,2020-04-26 12:00:00.9366667,2020-04-26 17:03:31.833333
1,2,9780001000452,GBP,5.99,BertramsInventory,2020-04-25 13:00:00.0317917,2020-04-26 17:03:31.833333
2,3,9780001837119,GBP,7.99,BertramsInventory,2020-04-25 13:00:00.0317917,2020-04-26 17:03:31.833333
3,4,9780001845633,GBP,7.99,GoldenList,2020-04-26 12:00:00.9366667,2020-04-26 17:03:31.833333
4,5,9780001982925,GBP,19.99,BertramsInventory,2020-04-25 13:00:00.0317917,2020-04-26 17:03:31.833333


In [5]:
list_price.shape

(1624530, 7)

In [6]:
list_price['PriceSource'].unique()

array(['GoldenList', 'BertramsInventory', 'BookDepotInventory',
       'BowkerNepModel', 'TitlePageScraper', 'NielsenFtp',
       'BowkerApiModel', 'PearsonCanadaScraperModel', 'SAPPurchaseOrder',
       'AbePolandScraper'], dtype=object)

In [7]:
list_price.drop(columns = ['Id', 'PriceSource', 'PriceLastRefreshed', 'Timestamp'], inplace = True)
list_price.head()

,Isbn,Currency,Price
0,9780000079145,GBP,67.00
1,9780001000452,GBP,5.99
2,9780001837119,GBP,7.99
3,9780001845633,GBP,7.99
4,9780001982925,GBP,19.99


In [8]:
bilblo_sql = """
SELECT *
FROM Isbn.Bibliography
"""
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

,Isbn,Title,TitleSource,TitleLastRefreshed,Publisher,PublisherSource,PublisherLastRefreshed,Author,AuthorSource,AuthorLastRefreshed,...,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp,Edition
0,9780000002402,None,None,None,LWW,GoldenList,2020-04-26 12:00:00.9366667,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None
1,9780000002679,None,None,None,LWW,GoldenList,2020-04-26 12:00:00.9366667,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None
2,9780000003102,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None
3,9780000003201,None,None,None,LWW,GoldenList,2020-04-26 12:00:00.9366667,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None
4,9780000006493,None,None,None,LWW,GoldenList,2020-04-26 12:00:00.9366667,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None


In [9]:
sup_list = pd.merge(list_price, biblio, how='left', on = 'Isbn' )

In [10]:
sup_list.head()

,Isbn,Currency,Price,Title,TitleSource,TitleLastRefreshed,Publisher,PublisherSource,PublisherLastRefreshed,Author,...,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp,Edition
0,9780000079145,GBP,67.00,None,None,None,LWW,GoldenList,2020-04-26 12:00:00.9366667,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None
1,9780001000452,GBP,5.99,Testy McTest and the Silver Bridle,BertramsInventory,2020-04-25 13:00:00.0317917,None,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None
2,9780001837119,GBP,7.99,Winter Story,BertramsInventory,2020-04-25 13:00:00.0317917,None,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None
3,9780001845633,GBP,7.99,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None
4,9780001982925,GBP,19.99,Collins Bedtime Treasury of Nursery Rhymes and...,BertramsInventory,2020-04-25 13:00:00.0317917,None,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-26 17:02:41.597,None


In [11]:
sup_list.columns

Index(['Isbn', 'Currency', 'Price', 'Title', 'TitleSource',
       'TitleLastRefreshed', 'Publisher', 'PublisherSource',
       'PublisherLastRefreshed', 'Author', 'AuthorSource',
       'AuthorLastRefreshed', 'PublicationDate', 'PublicationDateSource',
       'PublicationDateLastRefreshed', 'Copyright', 'CopyrightSource',
       'CopyrightLastRefreshed', 'Binding', 'BindingSource',
       'BindingLastRefreshed', 'Timestamp', 'Edition'],
      dtype='object')

In [12]:
sup_list = sup_list[['Isbn','Currency', 'Price','Title', 'Publisher', 'Author']]

In [13]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780000079145,GBP,67.00,None,LWW,None
1,9780001000452,GBP,5.99,Testy McTest and the Silver Bridle,None,None
2,9780001837119,GBP,7.99,Winter Story,None,None
3,9780001845633,GBP,7.99,None,None,None
4,9780001982925,GBP,19.99,Collins Bedtime Treasury of Nursery Rhymes and...,None,None


In [14]:
sup_list['Publisher'].unique()

array(['LWW', None, 'HARPER COLLINS', ..., 'Mandrake of Oxford', 'LOTUS',
       'Mandrake'], dtype=object)

In [15]:
sup_list = sup_list.fillna('N/A')
sup_list.isna().sum()

Isbn         0
Currency     0
Price        0
Title        0
Publisher    0
Author       0
dtype: int64

In [16]:
# 3. Clean publisher
dictio_sql = """
SELECT *
FROM dbo.PublisherDictionary
"""

dictio = pd.read_sql(dictio_sql, prouc_db)
dictio.head()

,PublisherLong,PublisherShort
0,"""Abrams, Inc.""",ABRAMS
1,"""Addison-Wesley Longman, Incorporated""",PEARSON
2,"""Addison-Wesley Longman, Limited""",PEARSON
3,"""Allyn & Bacon, Incorporated""",PEARSON
4,"""American Occupational Therapy Association, In...","American Occupational Therapy Association, Inc..."


In [17]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [18]:
for col in list(dictio.columns):
    dictio[col] = dictio.apply(lambda x: clean_up(x[col]), axis =1)

In [19]:
dictio.head()

,PublisherLong,PublisherShort
0,"""ABRAMS, INC.""",ABRAMS
1,"""ADDISON-WESLEY LONGMAN, INCORPORATED""",PEARSON
2,"""ADDISON-WESLEY LONGMAN, LIMITED""",PEARSON
3,"""ALLYN & BACON, INCORPORATED""",PEARSON
4,"""AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, IN...","AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, INC..."


In [20]:
for col in list(sup_list.drop(columns = 'Price').columns):
    sup_list[col] = sup_list.apply(lambda x: clean_up(x[col]), axis =1)

In [21]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780000079145,GBP,67.00,N/A,LWW,N/A
1,9780001000452,GBP,5.99,TESTY MCTEST AND THE SILVER BRIDLE,N/A,N/A
2,9780001837119,GBP,7.99,WINTER STORY,N/A,N/A
3,9780001845633,GBP,7.99,N/A,N/A,N/A
4,9780001982925,GBP,19.99,COLLINS BEDTIME TREASURY OF NURSERY RHYMES AND...,N/A,N/A


In [22]:
# Merge on Publisher
sup_list2 = pd.merge(sup_list, dictio, how='left', left_on = 'Publisher', right_on = 'PublisherLong' )

In [23]:
sup_list2.head()

,Isbn,Currency,Price,Title,Publisher,Author,PublisherLong,PublisherShort
0,9780000079145,GBP,67.00,N/A,LWW,N/A,LWW,LWW
1,9780001000452,GBP,5.99,TESTY MCTEST AND THE SILVER BRIDLE,N/A,N/A,N/A,N/A
2,9780001837119,GBP,7.99,WINTER STORY,N/A,N/A,N/A,N/A
3,9780001845633,GBP,7.99,N/A,N/A,N/A,N/A,N/A
4,9780001982925,GBP,19.99,COLLINS BEDTIME TREASURY OF NURSERY RHYMES AND...,N/A,N/A,N/A,N/A


In [24]:
sup_list2['Currency'].unique()

array(['GBP', 'USD', 'CND', 'AUD', 'NZD', 'EURO', 'PLN'], dtype=object)

In [25]:
# 4. get suppliers discount
sup_pub = pd.read_excel('../../../../Vendors/.  Vendors Details\supplier_procurement_details.xlsx',  sheet_name='pub')
sup_pub.head()

,Supplier,Publisher,Currency,Discount,Shipping Cost,max quantity,Min,Region
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUSTRALIA
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUSTRALIA
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUSTRALIA
3,COINFO,ALIEN & UNWIN,AUD,0.30,3.3,10000,0,AUSTRALIA
4,COINFO,Amer Psych Ass.,AUD,0.33,3.3,10000,0,AUSTRALIA


In [26]:
for col in ['Supplier', 'Publisher', 'Currency', 'Region']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)

In [27]:
sup_list3 = pd.merge(sup_list2, sup_pub, how='inner', left_on = ['Currency', 'PublisherShort'], right_on = ['Currency', 'Publisher'] )

In [28]:
sup_list3

,Isbn,Currency,Price,Title,Publisher_x,Author,PublisherLong,PublisherShort,Supplier,Publisher_y,Discount,Shipping Cost,max quantity,Min,Region
0,9780000079145,GBP,67.00,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
1,9780000003201,GBP,265.00,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
2,9780000002402,GBP,306.00,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
3,9780000002679,GBP,425.00,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
4,9780000050090,GBP,17.99,N/A,LWW,N/A,LWW,LWW,ABE,LWW,0.55,3.47,10000,0,EUROPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408926,9789027252722,AUD,78.95,"PIDGINS, CREOLES AND MIXED LANGUAGES: AN INTRO...",JOHN BENJAMINS PUBLISHING,VIVEKA VELUPILLAI,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
408927,9789027212092,AUD,79.95,HISTORY OF THE ENGLISH LANGUAGE,JOHN BENJAMINS PUBLISHING,ELLY VAN GELDEREN,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
408928,9789027213273,AUD,78.95,PRONUNCIATION FUNDAMENTALS,JOHN BENJAMINS PUBLISHING,TRACEY M. DERWING; MURRAY J. MUNRO,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
408929,9789027219060,AUD,75.95,COGNITIVE EXPLORATION OF LANGUAGE AND LINGUISTICS,JOHN BENJAMINS PUBLISHING,RENE DIRVEN,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA


In [29]:
sup_list3.drop(columns = ['Publisher_x', 'PublisherLong', 'Publisher_y'], inplace = True)

In [30]:
sup_list3.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region
0,9780000079145,GBP,67.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE
1,9780000003201,GBP,265.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE
2,9780000002402,GBP,306.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE
3,9780000002679,GBP,425.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE
4,9780000050090,GBP,17.99,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE


In [31]:
# 5. Check Suppliers special discounts
sp_disc = pd.read_excel("../../../../Vendors/. Short Discounted/all_short_discount_list.xlsx")

In [32]:
sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [33]:
sp_disc.dtypes

SUPPLIER     object
ISBN          int64
DISCOUNT    float64
dtype: object

In [34]:
sp_disc['ISBN'] = sp_disc['ISBN'].astype(str)
sp_disc.dtypes

SUPPLIER     object
ISBN         object
DISCOUNT    float64
dtype: object

In [35]:
for col in ['SUPPLIER', 'ISBN']:
    sp_disc[col] = sp_disc.apply(lambda x: clean_up(x[col]), axis =1)

sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [36]:
sup_list4 = pd.merge(sup_list3, sp_disc, how='left', left_on = ['Supplier', 'Isbn'], right_on = ['SUPPLIER', 'ISBN'] )

In [37]:
sup_list4.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT
0,9780000079145,GBP,67.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
1,9780000003201,GBP,265.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
2,9780000002402,GBP,306.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
3,9780000002679,GBP,425.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
4,9780000050090,GBP,17.99,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN


In [38]:
sup_list4[sup_list4['SUPPLIER']=='ABE']

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT
38,9780781750554,GBP,48.5,APPLIED PHLEBOTOMY,DENNIS J. ERNST,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781750554,0.35
60,9780781763103,GBP,59.0,PHARMACEUTICAL CALCULATIONS FOR THE PHARMACY T...,BARBARA LACHER,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781763103,0.35
69,9780781768702,GBP,55.0,ACSM'S RESOURCES FOR CLINICAL EXERCISE PHYSIOLOGY,AMERICAN COLLEGE OF SPORTS MEDICINE,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781768702,0.35
113,9780781775663,GBP,31.5,STUDY GUIDE FOR PHARMACOLOGY FOR HEALTH PROFES...,RENEE ACOSTA,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781775663,0.35
129,9780781782678,GBP,68.0,ACSM'S FOUNDATIONS OF STRENGTH TRAINING AND CO...,AMERICAN COLLEGE OF SPORTS MEDICINE (ACSM) STA...,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781782678,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405765,9781975123895,GBP,270.0,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9781975123895,0.35
405767,9781975127152,GBP,158.0,ORTHOPAEDIC KNOWLEDGE UPDATE: FOOT AND ANKLE 6...,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9781975127152,0.35
405790,9781975122676,GBP,240.0,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9781975122676,0.35
405791,9781975123697,GBP,270.0,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9781975123697,0.35


In [39]:
sup_list4.isna().sum()

Isbn                   0
Currency               0
Price                  0
Title                  0
Author                 0
PublisherShort         0
Supplier               0
Discount               0
Shipping Cost          0
max quantity           0
Min                    0
Region                 0
SUPPLIER          407448
ISBN              407448
DISCOUNT          407448
dtype: int64

In [40]:
sup_list4['final_discount'] = np.where(sup_list4['DISCOUNT'].isna(), sup_list4['Discount'],sup_list4['DISCOUNT'])

In [41]:
sup_list4.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT,final_discount
0,9780000079145,GBP,67.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
1,9780000003201,GBP,265.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
2,9780000002402,GBP,306.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
3,9780000002679,GBP,425.00,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
4,9780000050090,GBP,17.99,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55


In [42]:
sup_list4[sup_list4['Isbn'] == '9780781762021']

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT,final_discount


In [43]:
sup_list4 = sup_list4[['Isbn','Title', 'Author', 'PublisherShort', 'Supplier','Currency', 'Price', 'final_discount' ,'Shipping Cost', 'max quantity', 'Min', 'Region']]

In [44]:
sup_list4.head()

,Isbn,Title,Author,PublisherShort,Supplier,Currency,Price,final_discount,Shipping Cost,max quantity,Min,Region
0,9780000079145,N/A,N/A,LWW,ABE,GBP,67.00,0.55,3.47,10000,0,EUROPE
1,9780000003201,N/A,N/A,LWW,ABE,GBP,265.00,0.55,3.47,10000,0,EUROPE
2,9780000002402,N/A,N/A,LWW,ABE,GBP,306.00,0.55,3.47,10000,0,EUROPE
3,9780000002679,N/A,N/A,LWW,ABE,GBP,425.00,0.55,3.47,10000,0,EUROPE
4,9780000050090,N/A,N/A,LWW,ABE,GBP,17.99,0.55,3.47,10000,0,EUROPE


In [45]:
sup_list4.columns = map(str.lower, sup_list4.columns)

In [46]:
sup_list4.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
0,9780000079145,N/A,N/A,LWW,ABE,GBP,67.00,0.55,3.47,10000,0,EUROPE
1,9780000003201,N/A,N/A,LWW,ABE,GBP,265.00,0.55,3.47,10000,0,EUROPE
2,9780000002402,N/A,N/A,LWW,ABE,GBP,306.00,0.55,3.47,10000,0,EUROPE
3,9780000002679,N/A,N/A,LWW,ABE,GBP,425.00,0.55,3.47,10000,0,EUROPE
4,9780000050090,N/A,N/A,LWW,ABE,GBP,17.99,0.55,3.47,10000,0,EUROPE


In [47]:
# Add western campus
# Shipping Cost = 2
# 0-150CAD 10% off
# 150-200CAD 12% off
# > 200CAD 14%

can_sup = sup_list2[sup_list2['Currency'] == 'CND']
can_sup.columns = map(str.lower, can_sup.columns)
can_sup['supplier'] = 'WC'
can_sup['final_discount'] = np.where(can_sup['price']<150, 0.1, np.where((can_sup['price'] >=150) & (can_sup['price']<200),0.12,0.14))
can_sup['max quantity'] = 10000
can_sup['region'] = 'NORTH AMERICA'
can_sup['shipping cost'] = 2
can_sup['min'] = 0
can_sup.head()

,isbn,currency,price,title,publisher,author,publisherlong,publishershort,supplier,final_discount,max quantity,region,shipping cost,min
22,9780006392828,CND,19.95,N/A,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
23,9780006392910,CND,18.95,N/A,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
24,9780006393146,CND,19.95,N/A,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
25,9780006393740,CND,16.99,N/A,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
26,9780006394914,CND,19.95,N/A,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0


In [48]:
can_sup = can_sup[['isbn', 'title', 'author', 'publishershort', 'supplier','currency','price','final_discount', 'shipping cost', 'max quantity', 'min', 'region']]
can_sup['currency'] = np.where(can_sup['currency'] == 'CND', 'CAD',0)

In [49]:
can_sup.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
22,9780006392828,N/A,N/A,N/A,WC,CAD,19.95,0.1,2,10000,0,NORTH AMERICA
23,9780006392910,N/A,N/A,N/A,WC,CAD,18.95,0.1,2,10000,0,NORTH AMERICA
24,9780006393146,N/A,N/A,N/A,WC,CAD,19.95,0.1,2,10000,0,NORTH AMERICA
25,9780006393740,N/A,N/A,N/A,WC,CAD,16.99,0.1,2,10000,0,NORTH AMERICA
26,9780006394914,N/A,N/A,N/A,WC,CAD,19.95,0.1,2,10000,0,NORTH AMERICA


In [50]:
sup_list5 = sup_list4.append(can_sup, ignore_index= True)
sup_list5.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
0,9780000079145,N/A,N/A,LWW,ABE,GBP,67.00,0.55,3.47,10000,0,EUROPE
1,9780000003201,N/A,N/A,LWW,ABE,GBP,265.00,0.55,3.47,10000,0,EUROPE
2,9780000002402,N/A,N/A,LWW,ABE,GBP,306.00,0.55,3.47,10000,0,EUROPE
3,9780000002679,N/A,N/A,LWW,ABE,GBP,425.00,0.55,3.47,10000,0,EUROPE
4,9780000050090,N/A,N/A,LWW,ABE,GBP,17.99,0.55,3.47,10000,0,EUROPE


In [51]:
sup_list5['currency'].unique()

array(['GBP', 'AUD', 'USD', 'CAD'], dtype=object)

In [52]:
# Exchange Rate
#We add the Exchange rate column
exchange_rate = """
SELECT *
FROM dbo.XChange
"""
exch_df = pd.read_sql(exchange_rate,prouc_db)


In [53]:
exch_df

,id,name,rate,updatedon
0,2,AUD,0.638690,2020-04-26 08:00:01.190
1,3,NZD,0.602000,2020-04-26 08:00:01.190
2,4,GBP,1.237001,2020-04-26 08:00:01.190
3,5,CAD,0.708855,2020-04-26 08:00:01.190
4,6,EUR,1.082050,2020-04-26 08:00:01.190
5,8,SGD,0.701900,2020-04-26 08:00:01.190
6,9,HKD,0.129014,2020-04-26 08:00:01.190
7,1,USD,1.000000,2020-04-26 08:00:01.190
8,10,TWD,0.033135,2020-04-26 08:00:01.190
9,11,CHF,1.022320,2020-04-26 08:00:01.190


In [54]:
sup_list6 = pd.merge(sup_list5, exch_df, how='left', left_on = 'currency', right_on = 'name' )

sup_list6['invoice_fee'] = np.where(
    sup_list6['supplier'] == 'COINFO', (-1*sup_list6['price']*(1-sup_list6['final_discount']))/11,
    np.where(
        (sup_list6['supplier'] == 'FELIPE') | (sup_list6['supplier'] == 'ARMANDO'),
    0.01,
    0
     ))
sup_list6['lp_fee'] = 0
sup_list6.rename(columns={'publishershort': 'pub', 'final_discount': 'discount'}, inplace = True)


In [55]:
sup_list6 = sup_list6[['isbn', 'title', 'author', 'pub', 'supplier','currency','rate','price','discount','invoice_fee','lp_fee','shipping cost', 'max quantity', 'min', 'region']]
sup_list6.head()

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,max quantity,min,region
0,9780000079145,N/A,N/A,LWW,ABE,GBP,1.237001,67.00,0.55,0.0,0,3.47,10000,0,EUROPE
1,9780000003201,N/A,N/A,LWW,ABE,GBP,1.237001,265.00,0.55,0.0,0,3.47,10000,0,EUROPE
2,9780000002402,N/A,N/A,LWW,ABE,GBP,1.237001,306.00,0.55,0.0,0,3.47,10000,0,EUROPE
3,9780000002679,N/A,N/A,LWW,ABE,GBP,1.237001,425.00,0.55,0.0,0,3.47,10000,0,EUROPE
4,9780000050090,N/A,N/A,LWW,ABE,GBP,1.237001,17.99,0.55,0.0,0,3.47,10000,0,EUROPE


In [56]:
sup_list6.to_csv('suppliers_excel.csv', index= False)

In [57]:
sup_list6[sup_list6['supplier'] == 'FELIPE']

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,max quantity,min,region
310962,9780060000448,MIDDLE RANGE THEORIES: APPLICATION TO NURSING ...,SANDRA J. PETERSON,LWW,FELIPE,USD,1.0,73.99,0.59,0.01,0,1.7,150,1,SOUTH AMERICA
310963,9780060000387,GERONTOLOGICAL NURSING,CHARLOTTE ELIOPOULOS,LWW,FELIPE,USD,1.0,79.99,0.59,0.01,0,1.7,150,1,SOUTH AMERICA
310964,9780060000424,THEORETICAL NURSING,AFAF IBRAHAM MELEIS,LWW,FELIPE,USD,1.0,89.99,0.59,0.01,0,1.7,150,1,SOUTH AMERICA
310965,9780316069908,EATING ANIMALS,JONATHAN SAFRAN FOER,LWW,FELIPE,USD,1.0,25.99,0.59,0.01,0,1.7,150,1,SOUTH AMERICA
310966,9780316017923,OUTLIERS,MALCOLM GLADWELL,LWW,FELIPE,USD,1.0,29.00,0.59,0.01,0,1.7,150,1,SOUTH AMERICA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406096,9789065449283,N/A,N/A,W KLUWER,FELIPE,USD,1.0,1525.00,0.62,0.01,0,1.7,150,1,SOUTH AMERICA
406098,9789065449429,N/A,N/A,W KLUWER,FELIPE,USD,1.0,1844.00,0.62,0.01,0,1.7,150,1,SOUTH AMERICA
406100,9789065449399,N/A,N/A,W KLUWER,FELIPE,USD,1.0,2877.00,0.62,0.01,0,1.7,150,1,SOUTH AMERICA
406102,9789065449436,N/A,N/A,W KLUWER,FELIPE,USD,1.0,2046.00,0.62,0.01,0,1.7,150,1,SOUTH AMERICA
